[index](./index.ipynb) | [yhoo-profile](./profile.ipynb) | [yhoo-financials](./financials.ipynb) | [yhoo-holders](./holders.ipynb) | [yhoo-options](./options.ipynb) | [yhoo-history](./history.ipynb)

# Virtual finance API

Aims to represent financial sites like `finance.yahoo.com` as a virtual REST-API by scraping information from these sites and ...

   + call endpoints like in a REST-API
   + let them return JSON data
   + optionally have a 2nd layer process the JSON, for instance to be compatible with *yfinance* or *yahoo_fin* or simply code your own
   
## This notebook demonstrates the use of *virtual_finance_api wrapper* and the *yfinance* compatibily

## Jupyter

To use this Jupyter notebook you need to install Jupyter:

```
    $ pip install jupyter
```

See, http://jupyter.org/install.html for all details regarding Jupyter.

When installed you can start Jupyter from the cloned repo directory:
```
    $ cd jupyter
    $ jupyter notebook
``` 

... will launch your brouwser with the notebook tree structure.

In [1]:
import virtual_finance_api as fa
import virtual_finance_api.endpoints.business_insider as bie
import json

## Logging

The *virtual_finance_api* has logging integrated. To use it simply import the logging package and configure it. The standard loglevels apply: DEBUG, INFO, WARNING, ERROR, CRITICAL.

The `virtual_finance_api` make use of the `requests` library, which also supports logging. The logging of requests will also show up in the logfile according the loglevel that is set.

In [2]:
import logging

logging.basicConfig(
    filename="/tmp/virtual_finance_api.log",
    level=logging.INFO,
    format='%(asctime)s [%(levelname)s] %(name)s : %(message)s',
)

## The client processing the requests ...


the `client` always returns the response of the request. That same response is also assigned to the request-instance itself.

```
   r = SomeRequest(...)
   rv = Client.request(r)
   rv == r.response  # is True ! (unless an exception occurred of course
```

In [3]:
client = fa.Client()

In [4]:
ticker = 'IBM'

In [5]:
r = bie.ISIN(params={'query': ticker})

In [6]:
display(client.request(r))

{'ticker': 'IBM', 'ISIN': 'US4592001014'}

## ask for a non-existing ticker

Lets try to get the ISIN for a non-existing ticker and extend the response handling to handle diffferent responses.

This example cleary shows the `virtual` part: the initial response to get the HTML data went fine, but if the data can't be found in that response, a final 404 status is returned, just like a REST-API server would have done.

In [7]:
TICKERS = ['IBM', 'NONEXIST', 'CSCO']

for ticker in TICKERS:
    r = bie.ISIN(params={'query': ticker})
    try:
        client.request(r)
    
    except fa.VirtualFinanceAPIError as err:
        if r.status_code == 404:
            print(f"Not found: {ticker}")

    else:
        print("ISIN for {ticker} is: {ISIN}".format(**r.response))

ISIN for IBM is: US4592001014
Not found: NONEXIST
ISIN for CSCO is: US17275R1023


# Simply the *yfinance* way ?

... in that case these 2 lines are enough (better is to extend with the logging enabled, see [Logging](#Logging))

In [8]:
import virtual_finance_api.compat.yfinance as yf

ticker = yf.Ticker('IBM')

In [9]:
display(ticker.history())

,Open,High,Low,Close,Volume
2021-03-01 14:30:00,120.349998,122.320000,119.860001,120.739998,5714500
2021-03-02 14:30:00,120.739998,121.900002,120.260002,120.330002,4522200
2021-03-03 14:30:00,120.500000,122.629997,119.980003,122.360001,7396200
2021-03-04 14:30:00,122.000000,123.220001,118.760002,120.110001,8062100
2021-03-05 14:30:00,120.639999,123.750000,120.250000,122.830002,6944900
2021-03-08 14:30:00,122.989998,126.849998,122.879997,124.809998,7236600
2021-03-09 14:30:00,125.400002,126.430000,124.160004,124.180000,5608200
2021-03-10 14:30:00,125.050003,128.240005,124.610001,127.870003,7243500
2021-03-11 14:30:00,128.089996,128.639999,126.779999,127.139999,5145000
2021-03-12 14:30:00,127.190002,127.680000,126.610001,127.610001,4009600


In [10]:
display(ticker.earnings)

,Revenue,Earnings
Year,,
2017,79139000000,5753000000
2018,79591000000,8728000000
2019,77147000000,9431000000
2020,73621000000,5590000000


In [11]:
display(ticker.financials)

,2020-12-31,2019-12-31,2018-12-31,2017-12-31
Research Development,6333000000.0,5936000000.0,5379000000.0,5590000000.0
Effect Of Accounting Charges,None,None,None,None
Income Before Tax,4637000000.0,10166000000.0,11342000000.0,11400000000.0
Minority Interest,129000000.0,144000000.0,134000000.0,131000000.0
Net Income,5590000000.0,9431000000.0,8728000000.0,5753000000.0
Selling General Administrative,21242000000.0,20382000000.0,19366000000.0,21137000000.0
Gross Profit,35575000000.0,36489000000.0,36937000000.0,36943000000.0
Ebit,8584000000.0,10785000000.0,13218000000.0,11682000000.0
Operating Income,8584000000.0,10785000000.0,13218000000.0,11682000000.0
Other Operating Expenses,-584000000.0,-614000000.0,-1026000000.0,-1466000000.0


[index](./index.ipynb) | [yhoo-profile](./profile.ipynb) | [yhoo-financials](./financials.ipynb) | [yhoo-holders](./holders.ipynb) | [yhoo-options](./options.ipynb) | [yhoo-history](./history.ipynb)